In [1]:
import openmc
import numpy as np

In [ ]:
#this is a model of a BCC lattice unit cell with a reflecting boundary

In [3]:
#materials
#these are functionally dummy values until you have a chance to look everything up.  note to self - include sources in comments.
#also doublecheck isotopic compostions conventions in mats
uco = openmc.Material(name='UCO')
uco.set_density('kg/m3', 1.0)
#you might also be able to do this more cleanly with the enrichment options add_element has
uco.add_nuclide('U235', 4.6716e-02, percent_type='ao')#'wo' is for weight percent
uco.add_nuclide('U238', 2.8697e-01, percent_type='ao')#'wo' is for weight percent
uco.add_nuclide('O16',  5.0000e-01, percent_type='ao')#'wo' is for weight percent
uco.add_element('C', 1.6667e-01)

buffer = openmc.Material(name='buffer')
buffer.set_density('kg/m3', 1.0)
buffer.add_element('C', 1.0)
buffer.add_s_alpha_beta('c_Graphite')

pyc = openmc.Material(name='PyC')
pyc.set_density('kg/m3', 1.0)
pyc.add_element('C', 1.0)
pyc.add_s_alpha_beta('c_Graphite')

sic = openmc.Material(name='SiC')
sic.set_density('kg/m3', 1.0)
sic.add_element('C', 0.5)
sic.add_element('Si', 0.5)
sic.add_s_alpha_beta('c_Graphite')

graphite = openmc.Material(name='graphite')
graphite.set_density('kg/m3', 1.0)
graphite.add_element('C', 1.0)
graphite.add_s_alpha_beta('c_Graphite')

he = openmc.Material(name='He')
he.set_density('atom/b-cm', 0.0006)
he.add_element('He', 1.0)

In [4]:
#geometry parameters - remember LAMMPS coords are in meters
#remember, triso order: kernel, buffer, pyc, sic, pyc
#fun and interesting note: openmc defaults to cm, and you cannot change that for geometry
#so you have to remember to convert LAMMPS later.  Also, these below are in meters so you gotta change these DO NOT FORGET
peb_or = 0.03 #outer radius of whole pebble
peb_ir = 0.025 # radius of the region that has trisos in it only
triso_r = [0.0002125, 0.0003125, 0.0003525, 0.0003875, 0.0004275]
cube_l = (4*peb_or)**(3)

In [ ]:
#define the cells and universe for the triso particles
triso_bounds = [openmc.Sphere(r=r) for r in triso_r]
triso_cells = [openmc.Cell(fill=uco, region=-spheres[0]),
               openmc.Cell(fill=buffer, region=+spheres[0] & -spheres[1]),
               openmc.Cell(fill=pyc, region=+spheres[1] & -spheres[2]),
               openmc.Cell(fill=sic, region=+spheres[2] & -spheres[3]),
               openmc.Cell(fill=pyc, region=+spheres[3])]
triso_univ = openmc.Universe(cells=triso_cells)

# define where the particles will be packed
# BCC lattice has one sphere at the center, and a 1/4 sphere at each corner, tight enough they are all touching.
#that part should be handled with your calculation of the cube length (the diameter = 2 peb diameters)

#start with the easy part: the center pebble

body_peb_in = openmc.Sphere(r = peb_ir)
body_peb_wfuel = -body_peb_in

#you should be able to pack the center and corners separately (and that might be easier than trying to define the corners as one big cell
#you have 8 corners.  looking at the cube with the nearest face towards you, we will start with the bottom-left corner
#and go clockwise around the bottom, then we will move to the top-left and move clockwise around the top

#a note: this is *specifically* the part of the corners that has the trisos in it.  so the center is at the corner, but it
#only goes out to peb_ir.

#start with getting the center coordinate

# generate triso particle centers

#put them in there